<a href="https://colab.research.google.com/github/morenourey/ENTREGABLES/blob/main/TFM_DATA_PREPOCESSING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 7.0 MB/s eta 0:00:00


In [46]:
import sqlite3 as sql
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns',None)

import warnings
warnings.filterwarnings("ignore")

In [3]:
# fechas
from datetime import datetime

# gráficos
import seaborn as sns
import matplotlib.pyplot as plt

# preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, Normalizer, MinMaxScaler
from sklearn.preprocessing import PolynomialFeatures

# algoritmos de clasificación
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

# algoritmos de regresión
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, SGDRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

# métricas
import sklearn.metrics as metrics

# hiperparametrización
from sklearn.model_selection import GridSearchCV

# TensorFlow y tf.keras
import tensorflow as tf
from tensorflow import keras

import xgboost as xgb

from numpy import random
random.seed(42)

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
df_product = pd.read_csv('/content/drive/MyDrive/NUCLIO/TFM_FINTEC/products_df.csv')

df_sociodemo = pd.read_csv('/content/drive/MyDrive/NUCLIO/TFM_FINTEC/sociodemographic_df.csv')

df_comercial = pd.read_csv('/content/drive/MyDrive/NUCLIO/TFM_FINTEC/commercial_activity_df.csv')

In [22]:
df_product.head(5)

,Unnamed: 0,pk_cid,pk_partition,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount
0,0,1375586,2018-01-28,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,1
1,1,1050611,2018-01-28,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,1
2,2,1050612,2018-01-28,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,1
3,3,1050613,2018-01-28,1,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0
4,4,1050614,2018-01-28,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,1


In [28]:
df_product['payroll'].value_counts(normalize=True)*100

payroll
0.0    96.548856
1.0     3.451144
Name: proportion, dtype: float64

In [30]:
df_product['pension_plan'].value_counts(normalize=True)*100

pension_plan
0.0    96.347359
1.0     3.652641
Name: proportion, dtype: float64

In [11]:
df_product.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5962924 entries, 0 to 5962923
Data columns (total 18 columns):
 #   Column              Dtype  
---  ------              -----  
 0   Unnamed: 0          int64  
 1   pk_cid              int64  
 2   pk_partition        object 
 3   short_term_deposit  int64  
 4   loans               int64  
 5   mortgage            int64  
 6   funds               int64  
 7   securities          int64  
 8   long_term_deposit   int64  
 9   em_account_pp       int64  
 10  credit_card         int64  
 11  payroll             float64
 12  pension_plan        float64
 13  payroll_account     int64  
 14  emc_account         int64  
 15  debit_card          int64  
 16  em_account_p        int64  
 17  em_acount           int64  
dtypes: float64(2), int64(15), object(1)
memory usage: 818.9+ MB


In [15]:
df_product.nunique()

Unnamed: 0            5962924
pk_cid                 456373
pk_partition               17
short_term_deposit          2
loans                       2
mortgage                    2
funds                       2
securities                  2
long_term_deposit           2
em_account_pp               1
credit_card                 2
payroll                     2
pension_plan                2
payroll_account             2
emc_account                 2
debit_card                  2
em_account_p                2
em_acount                   2
dtype: int64

In [16]:
df_product.isnull().sum()

Unnamed: 0             0
pk_cid                 0
pk_partition           0
short_term_deposit     0
loans                  0
mortgage               0
funds                  0
securities             0
long_term_deposit      0
em_account_pp          0
credit_card            0
payroll               61
pension_plan          61
payroll_account        0
emc_account            0
debit_card             0
em_account_p           0
em_acount              0
dtype: int64

In [23]:
df_sociodemo.head()

,Unnamed: 0,pk_cid,pk_partition,country_id,region_code,gender,age,deceased,salary
0,0,1375586,2018-01-28,ES,29.0,H,35,N,87218.10
1,1,1050611,2018-01-28,ES,13.0,V,23,N,35548.74
2,2,1050612,2018-01-28,ES,13.0,V,23,N,122179.11
3,3,1050613,2018-01-28,ES,50.0,H,22,N,119775.54
4,4,1050614,2018-01-28,ES,50.0,V,23,N,NaN


In [26]:
df_sociodemo['country_id'].value_counts(normalize=True)*100

country_id
ES    99.962233
GB     0.007396
FR     0.003773
DE     0.003337
US     0.003270
CH     0.003253
BR     0.001459
BE     0.001358
VE     0.001325
IE     0.001140
MX     0.000973
AT     0.000855
AR     0.000855
PL     0.000822
IT     0.000755
MA     0.000570
CL     0.000503
CN     0.000470
CA     0.000369
LU     0.000285
ET     0.000285
QA     0.000285
CI     0.000285
SA     0.000285
CM     0.000285
SN     0.000285
MR     0.000285
NO     0.000285
RU     0.000285
CO     0.000285
GA     0.000285
GT     0.000285
DO     0.000285
SE     0.000268
DJ     0.000184
PT     0.000184
JM     0.000184
RO     0.000151
HU     0.000134
DZ     0.000117
PE     0.000067
Name: proportion, dtype: float64

In [49]:
df_sociodemo['region_code'].value_counts(normalize=True).cumsum()*100

region_code
28.0     19.893049
8.0      29.965121
46.0     35.973550
30.0     40.953804
41.0     45.826536
15.0     50.102757
29.0     53.255277
36.0     56.205454
3.0      59.080270
11.0     61.779534
33.0     64.037036
50.0     66.268047
6.0      68.324649
35.0     70.371100
47.0     72.333097
18.0     74.057554
45.0     75.774277
37.0     77.326756
10.0     78.723967
14.0     79.989011
2.0      81.198122
21.0     82.406663
13.0     83.582573
39.0     84.750044
12.0     85.902752
7.0      86.950220
32.0     87.866881
27.0     88.752403
43.0     89.621133
17.0     90.472532
9.0      91.273550
25.0     92.060158
16.0     92.665695
24.0     93.260025
48.0     93.851503
26.0     94.435163
4.0      94.983425
31.0     95.507544
49.0     96.027034
23.0     96.546356
38.0     97.032896
34.0     97.495009
19.0     97.943986
40.0     98.307335
5.0      98.664678
22.0     99.018867
20.0     99.334822
1.0      99.552902
44.0     99.764556
42.0     99.907158
51.0     99.955743
52.0    100.000000


In [51]:
df_sociodemo['region_code'].value_counts(normalize=True)*100

region_code
28.0    19.893049
8.0     10.072073
46.0     6.008429
30.0     4.980254
41.0     4.872732
15.0     4.276221
29.0     3.152520
36.0     2.950177
3.0      2.874816
11.0     2.699265
33.0     2.257502
50.0     2.231011
6.0      2.056601
35.0     2.046451
47.0     1.961997
18.0     1.724457
45.0     1.716723
37.0     1.552479
10.0     1.397211
14.0     1.265044
2.0      1.209111
21.0     1.208541
13.0     1.175910
39.0     1.167471
12.0     1.152708
7.0      1.047468
32.0     0.916660
27.0     0.885523
43.0     0.868729
17.0     0.851399
9.0      0.801019
25.0     0.786608
16.0     0.605537
24.0     0.594330
48.0     0.591478
26.0     0.583660
4.0      0.548261
31.0     0.524120
49.0     0.519489
23.0     0.519322
38.0     0.486540
34.0     0.462113
19.0     0.448977
40.0     0.363349
5.0      0.357343
22.0     0.354189
20.0     0.315955
1.0      0.218080
44.0     0.211654
42.0     0.142602
51.0     0.048585
52.0     0.044257
Name: proportion, dtype: float64

In [12]:
df_sociodemo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5962924 entries, 0 to 5962923
Data columns (total 9 columns):
 #   Column        Dtype  
---  ------        -----  
 0   Unnamed: 0    int64  
 1   pk_cid        int64  
 2   pk_partition  object 
 3   country_id    object 
 4   region_code   float64
 5   gender        object 
 6   age           int64  
 7   deceased      object 
 8   salary        float64
dtypes: float64(2), int64(3), object(4)
memory usage: 409.4+ MB


In [17]:
df_sociodemo.nunique()

Unnamed: 0      5962924
pk_cid           456373
pk_partition         17
country_id           41
region_code          52
gender                2
age                 104
deceased              2
salary           258629
dtype: int64

In [18]:
df_sociodemo.isnull().sum()

Unnamed: 0            0
pk_cid                0
pk_partition          0
country_id            0
region_code        2264
gender               25
age                   0
deceased              0
salary          1512103
dtype: int64

In [31]:
df_comercial.head()

,Unnamed: 0,pk_cid,pk_partition,entry_date,entry_channel,active_customer,segment
0,0,1375586,2018-01-28,2018-01-12,KHL,1.0,02 - PARTICULARES
1,1,1050611,2018-01-28,2015-08-10,KHE,0.0,03 - UNIVERSITARIO
2,2,1050612,2018-01-28,2015-08-10,KHE,0.0,03 - UNIVERSITARIO
3,3,1050613,2018-01-28,2015-08-10,KHD,0.0,03 - UNIVERSITARIO
4,4,1050614,2018-01-28,2015-08-10,KHE,1.0,03 - UNIVERSITARIO


In [32]:
df_comercial['segment'].value_counts(normalize=True)*100

segment
03 - UNIVERSITARIO    66.909922
02 - PARTICULARES     31.409869
01 - TOP               1.680208
Name: proportion, dtype: float64

In [33]:
df_comercial['entry_channel'].value_counts(normalize=True)*100

entry_channel
KHE    53.413469
KFC    15.276786
KHQ    10.125061
KAT     7.137080
KHK     3.948564
         ...    
KEJ     0.000137
KHS     0.000086
KDA     0.000034
KFP     0.000034
KDS     0.000017
Name: proportion, Length: 68, dtype: float64

In [34]:
df_comercial['entry_date'].value_counts(normalize=True)*100

entry_date
2017-07-28    0.962431
2017-10-03    0.910409
2017-08-04    0.767174
2016-10-14    0.684295
2016-08-03    0.560363
                ...   
2015-04-29    0.000184
2016-06-15    0.000184
2015-08-19    0.000184
2018-05-31    0.000151
2017-05-01    0.000117
Name: proportion, Length: 1499, dtype: float64

In [13]:
df_comercial.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5962924 entries, 0 to 5962923
Data columns (total 7 columns):
 #   Column           Dtype  
---  ------           -----  
 0   Unnamed: 0       int64  
 1   pk_cid           int64  
 2   pk_partition     object 
 3   entry_date       object 
 4   entry_channel    object 
 5   active_customer  float64
 6   segment          object 
dtypes: float64(1), int64(2), object(4)
memory usage: 318.5+ MB


In [19]:
df_comercial.nunique()

Unnamed: 0         5962924
pk_cid              456373
pk_partition            17
entry_date            1499
entry_channel           68
active_customer          2
segment                  3
dtype: int64

In [20]:
df_comercial.isnull().sum()

Unnamed: 0              0
pk_cid                  0
pk_partition            0
entry_date              0
entry_channel      133033
active_customer         0
segment            133944
dtype: int64

In [43]:
df_comercial['entry_channel'].value_counts(normalize = True).cumsum()*100

entry_channel
KHE     53.413469
KFC     68.690255
KHQ     78.815316
KAT     85.952396
KHK     89.900960
          ...    
KEJ     99.999828
KHS     99.999914
KDA     99.999949
KFP     99.999983
KDS    100.000000
Name: proportion, Length: 68, dtype: float64

In [1]:
df_comercial_product = pd.merge(df_comercial, df_product, on='pk_cid', how='outer')
df_comercial_product

NameError: name 'pd' is not defined